# Conditional value at risk in robust portfolio

In [1]:
import cvxpy as cp
import numpy as np
import torch
import matplotlib.pyplot as plt
import pandas as pd
import yfinance as yf


import lropt
from lropt.parameter import Parameter
from lropt.robust_problem import RobustProblem
from lropt.uncertain import UncertainParameter

np.random.seed(seed=1234)

In [2]:
stocks = ['JPM', 'AMZN', 'TSLA', 'AAPL', 'GOOG', 'ZM', 'META', 'MCD']
start = '2021-1-2'              # starting date of historical data
end='2021-12-31'                # end date of historical data

data = pd.DataFrame([])
for stock in stocks:
    each = yf.Ticker(stock).history(start=start, end=end)
    close = each['Close'].values
    returns = (close[1:] - close[:-1]) / close[:-1]
    data[stock] = returns

data

,JPM,AMZN,TSLA,AAPL,GOOG,ZM,META,MCD
0,0.005441,0.010004,0.007317,0.012364,0.007337,0.002361,0.007548,0.005994
1,0.046956,-0.024897,0.028390,-0.033662,-0.003234,-0.045506,-0.028269,-0.002270
2,0.032839,0.007577,0.079447,0.034123,0.029943,-0.005546,0.020622,0.004644
3,0.001104,0.006496,0.078403,0.008632,0.011168,0.020759,-0.004354,0.018351
4,0.014924,-0.021519,-0.078214,-0.023249,-0.022405,-0.034038,-0.040102,-0.007597
...,...,...,...,...,...,...,...,...
245,0.003574,0.000184,0.057619,0.003644,0.001317,-0.007663,0.014495,0.003812
246,0.005723,-0.008178,0.025248,0.022975,0.006263,-0.021967,0.032633,0.008611
247,0.003035,0.005844,-0.005000,-0.005767,-0.010914,-0.019580,0.000116,-0.001342
248,-0.000504,-0.008555,-0.002095,0.000502,0.000386,-0.010666,-0.009474,0.002277


In [4]:
y = data.values     # stock data as an array
s, n = y.shape      # s: sample size; n: number of stocks

x_lb = np.zeros(n)  # lower bounds of investment decisions
x_ub = np.ones(n)   # upper bounds of investment decisions

beta =0.95          # confidence interval
w0 = 1              # investment budget
mu = 0.001          # target minimum expected return rate

eta_ub = 0.0001     # upper bound of eta
eta_lb = -0.0001    # lower bound of eta

In [15]:
uncertainty_set = lropt.Box(rho=max(eta_ub, -eta_lb), lb=eta_lb, ub=eta_ub, eq=0)
eta = UncertainParameter(s, uncertainty_set=uncertainty_set)
pi = 1/s + eta
x = cp.Variable(n)
u = cp.Variable(s)
alpha = cp.Variable()

objective = cp.Minimize(alpha + 1/(1-beta) * (pi@u))
constraints = [
                u >= -y@x - alpha,
                u >= 0,
                pi@(y@x) >= mu,
                x >= x_lb,
                x <= x_ub,
                cp.sum(x) == w0,
              ]
prob = RobustProblem(objective=objective, constraints=constraints)

In [18]:
prob.solve()

0.018363501796245556

# Joint Production-Inventory

# Adaptive robust lot sizing